<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Web Scraping

_Author: Dave Yerrington (SF)_

---

## Before Class

#### Install Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

To install Selenium, use one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`


#### Install GeckoDriver

You will also need GeckoDriver (this assumes you are using Homebrew for Mac): 

- ```brew install geckodriver```

#### Install Firefox

Additionally, you will need to have downloaded the [Firefox browser](https://www.mozilla.org/en-US/firefox/new/?utm_source=google&utm_medium=cpc&utm_campaign=Firefox-Brand-US-GGL-Exact&utm_term=firefox&utm_content=A144_A203_A006336&gclid=Cj0KEQjwnPLKBRC-j7nt1b7OlZwBEiQAv8lMLJUyReT6cPzSYdmEA6uD3YDoieuuuusddgAU7XH6smEaAoje8P8HAQ&gclsrc=aw.ds) for the application in this lesson.

## Learning Objectives
- Revisit how to locate elements on a webpage
- Aquire unstructure data from the internet using Beautiful soup.
- Discuss limitations associated with simple requests and urllib libraries
- Introduce Selenium as a solution, and implement a scraper using selenium

## Lesson Guide

- [Introduction](#intro)
- [Building a web scraper](#building-scraper)
- [Retrieving data from the HTML page](#retrieving-data)
    - [Retrieving the restaurant names](#retrieving-names)
    - [Challenge: Retrieving the restaurant locations](#retrieving-locations)
    - [Retrieving the restaurant prices](#retrieving-prices)
    - [Retrieving the restaurant number of bookings](#retrieving-bookings)


- [Introducting Selenium](#selenium)
    - [Running JavaScript before scraping](#selenium-js)
    - [Using regex to only get digits](#selenium-regex)
    - [Challenge: Use Pandas to create a DataFrame of bookings](#challenge-pandas)
    - [Auto-typing using Selenium](#selenium-typing)


- [Summary](#summary)

<a id="intro"></a>
## Introduction

In this codealong lesson, we'll build a web scraper using urllib and BeautifulSoup. We will also explore how to use a headless browser called Selenium.

We'll begin by scraping OpenTable's DC listings. We're interested in knowing the restaurant's **name, location, price, and how many people booked it today.**

OpenTable provides all of this information on this given page: http://www.opentable.com/washington-dc-restaurant-listings

Let's inspect the elements of this page to assure we can find each of the bits of information in which we're interested.

---

<a id="building-scraper"></a>
## Building a web scraper

Now, let's build a web scraper for OpenTable using urllib and Beautiful Soup:

In [1]:
# import our necessary first packages
from bs4 import BeautifulSoup
import urllib

In [2]:
# set the url we want to visit
url = "http://www.opentable.com/washington-dc-restaurant-listings"

# visit that url, and grab the html of said page
html = urllib.urlopen(url).read()

At this point, what is in html?

In [3]:
html[:500]

'          <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/> <title>Washington, D.C. Area Restaurants List | OpenTable</title>  <meta  name="description" content="Find Washington, D.C. Area restaurants. Search by location, cuisine, or price to refine restaurant results in the Washington, D.C. Area area." > </meta>  <meta  name="robots" content="noindex" > </meta>     <link rel="shortcut icon" href="//components.ots'

We will need to convert this html objct into a soup object so we can parse it using python and BS4

In [4]:
# convert this into a soup object
soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")

<a id="retrieving-data"></a>
### Retrieving data from the HTML page

Let's first find each restaurant name listed on the page we've loaded. How do we find the page location of the restaurant? (Hint: We need to know where in the **HTML** the restaurant element is housed.) In order to find the HTML that renders the restaurant location, we can use Google Chrome's Inspect tool:

> http://www.opentable.com/washington-dc-restaurant-listings

> 1. Visit the URL above. 

> 2. Right-click on an element you are interested in, then choose Inspect (in Chrome). 

> 3. This will open the Developer Tools and show the HTML used to render the selected page element. 

> Throughout this lesson, we will use this method to find tags associated with elements of the page we want to scrape.

See if you can find the restaurant name on the page. Keep in mind there are many restaurants loaded on the page.

In [5]:
# print the restaurant names
soup.find_all(name='span', attrs={'class':'rest-row-name-text'})

[<span class="rest-row-name-text">Ruffino's - Arlington</span>,
 <span class="rest-row-name-text">Joe's Place Pizza and Pasta</span>,
 <span class="rest-row-name-text">Founding Farmers - DC</span>,
 <span class="rest-row-name-text">Farmers Fishers Bakers</span>,
 <span class="rest-row-name-text">Filomena Ristorante</span>,
 <span class="rest-row-name-text">Ambar - Arlington</span>,
 <span class="rest-row-name-text">Rasika West End</span>,
 <span class="rest-row-name-text">Blue Duck Tavern</span>,
 <span class="rest-row-name-text">Founding Farmers - Tysons</span>,
 <span class="rest-row-name-text">BlackSalt</span>,
 <span class="rest-row-name-text">Kapnos Taverna Arlington</span>,
 <span class="rest-row-name-text">Et Voila</span>,
 <span class="rest-row-name-text">Tupelo Honey - Arlington</span>,
 <span class="rest-row-name-text">SER</span>,
 <span class="rest-row-name-text">The Melting Pot - Arlington VA</span>,
 <span class="rest-row-name-text">1789 Restaurant</span>,
 <span class="re

It is important to always keep in mind the data types that were returned. Note this is a `list`, and we know that immediately by observing the outer square brackets and commas separating each tag.

Next, note the elements of the list are `Tag` objects, not strings. (If they were strings, they would be surrounded by quotes.) The Beautiful Soup authors chose to display a `Tag` object visually as a text representation of the tag and its contents. However, being an object, it has many methods that we can call on it. For example, next we will use the `encode_contents()` method to return the tag's contents encoded as a Python string.

<a id="retrieving-names"></a>
#### Retrieving the restaurant names

Now that we found a list of tags containing the restaurant names, let's think how we can loop through them all one-by-one. In the following cell, we'll print out the name (and **only** the clean name, not the rest of the html) of each restaurant.

In [6]:
# for each element you find, print out the restaurant name
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    print entry.encode_contents()

Ruffino's - Arlington
Joe's Place Pizza and Pasta
Founding Farmers - DC
Farmers Fishers Bakers
Filomena Ristorante
Ambar - Arlington
Rasika West End
Blue Duck Tavern
Founding Farmers - Tysons
BlackSalt
Kapnos Taverna Arlington
Et Voila
Tupelo Honey - Arlington
SER
The Melting Pot - Arlington VA
1789 Restaurant
The Liberty Tavern
Chez Billy Sud
Ray's The Steaks
Lyon Hall
Bistro Aracosia
Baba
Green Pig Bistro
Nobu DC
CIRCA at Clarendon
Eddie V's - Tysons Corner
Il Canale
Bourbon Steak - Four Seasons Washington DC
J. Gilbert’s – Wood Fired Steaks &amp; Seafood - McLean
Clyde's of Georgetown
Boqueria - DC
Café Milano
Uncle Julio's - Arlington/ Ballston
District Commons
Rustico - Ballston
Texas Jacks Barbecue
Honeysuckle
Mussel Bar &amp; Grille - Arlington
Kingbird at The Watergate Hotel
Cheesetique - Ballston
Barley Mac
El Centro D.F. - Georgetown
La Cote D'Or
CIRCA at Foggy Bottom
Seasons 52 - Tysons Corner
Pamplona
Yayla Bistro
Peacock Café
Grand Cru Wine Bar &amp; Bistro
The Capital Gri

Great!

<a id="retrieving-locations"></a>
#### Challenge: Retrieving the restaurant locations

Can you repeat that process for finding the location? For example, barmini by Jose Andres is in the location listed as "Penn Quarter" in our search results.

In [51]:
# first, see if you can identify the location for all elements -- print it out
# soup.find_all(name='span', attrs={'class':'rest-row-meta--location'})

In [53]:
# now print out EACH location for the restaurants
for entry in soup.find_all(name='span', attrs={'class':['rest-row-meta--location', 'rest-row-name-text']}):
    print entry.encode_contents()

Ruffino's - Arlington
Arlington
Joe's Place Pizza and Pasta
Arlington
Founding Farmers - DC
Foggy Bottom
Farmers Fishers Bakers
Georgetown
Filomena Ristorante
Georgetown
Ambar - Arlington
Arlington
Rasika West End
West End
Blue Duck Tavern
West End
Founding Farmers - Tysons
Tysons Corner / McLean
BlackSalt
Palisades Northwest
Kapnos Taverna Arlington
Arlington
Et Voila
Palisades Northwest
Tupelo Honey - Arlington
Arlington
SER
Arlington
The Melting Pot - Arlington VA
Arlington
1789 Restaurant
Georgetown
The Liberty Tavern
Arlington
Chez Billy Sud
Georgetown
Ray's The Steaks
Arlington
Lyon Hall
Arlington
Bistro Aracosia
Palisades Northwest
Baba
Arlington
Green Pig Bistro
Arlington
Nobu DC
Washington
CIRCA at Clarendon
Arlington
Eddie V's - Tysons Corner
Tysons Corner / McLean
Il Canale
Georgetown
Bourbon Steak - Four Seasons Washington DC
Georgetown
J. Gilbert’s – Wood Fired Steaks &amp; Seafood - McLean
McLean
Clyde's of Georgetown
Georgetown
Boqueria - DC
Dupont Circle
Café Milano
Geo

<a id="retrieving-prices"></a>
#### Retrieving the restaurant prices

Ok, we've figured out the restaurant name and location. Now we need to grab the price (number of dollar signs on a scale of one to four) for each restaurant. We'll follow the same process.

In [9]:
# print out all prices
soup.find_all('div', {'class':'rest-row-pricing'})

[<div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pri

In [10]:
# print out EACH number of dollar signs per restaurant
# this one is trickier to eliminate the html. Hint: try a nested find
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    print entry.find('i').encode_contents()

  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    
  $    $      
  $    $    $    
  $    $    $    
  $    $      
  $    $    $    
  $    $    $    
  $    $      
  $    $      
  $    $      
  $    $    $    
  $    $    $    $  
  $    $      
  $    $      
  $    $    $    
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $      
  $    $    $    
  $    $      
  $    $    $    $  
  $    $    $    
  $    $      
  $    $    $    
  $    $    $    
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $    $    
  $    $    $    
  $    $    $    $  
  $    $      
  $    $      
 

That looks great, but what if I wanted just the number of dollar signs per restaurant? Can you figure out a way to simply print out the number of dollar signs per restaurant listed?

In [61]:
# print the number of dollars signs per restaurant
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    print len(entry.find('i').encode_contents().split())

2
2
2
2
3
2
3
3
2
3
3
2
2
2
3
4
2
2
3
2
2
2
2
4
2
3
2
4
3
2
3
3
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
4
2
2
2
4
4
2
3
3
4
2
2
2
2
4
2
2
2
3
2
2
2
2
4
2
3
4
2
2
2
2
2
3
2
2
3
2
2
3
3
4
2
2
2
4
3
4
3
2
2
2


Phew, nice work. 

<a id="retrieving-bookings"></a>
#### Retrieving the restaurant number of bookings

One more, right? We only need to find the number times a restaurant was booked. In the next cell, print out all objects that contain the number of times the restaurant was booked.

In [62]:
# print out all objects that contain the number of times the restaurant was booked
soup.find_all('div', {'class':'booking'})

[]

That's weird -- an empty set. Did we find the wrong element? What's going on here? Discuss.

How can we debug this? Any ideas?

In [13]:
# let's first try printing out all 'div' objects
#  NOTE: This is a too many objects to store in this notebook!
#        So, uncomment the code below to run it.

# for entry in soup.find_all('div'):
#     print entry

I still don't see it. Let's search our entire soup object:

In [14]:
# print out soup, do command+f for "booked ". 
#   Uncomment the below to run.

# soup

What do you notice? Why is this happening?

<a id="selenium"></a>
## Introducing Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

To install Selenium, use one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`

You will also need GeckoDriver (this assumes you are using Homebrew for Mac): 

- ```brew install geckodriver```

Additionally, you will need to have downloaded the [Firefox browser](https://www.mozilla.org/en-US/firefox/new/?utm_source=google&utm_medium=cpc&utm_campaign=Firefox-Brand-US-GGL-Exact&utm_term=firefox&utm_content=A144_A203_A006336&gclid=Cj0KEQjwnPLKBRC-j7nt1b7OlZwBEiQAv8lMLJUyReT6cPzSYdmEA6uD3YDoieuuuusddgAU7XH6smEaAoje8P8HAQ&gclsrc=aw.ds) for the application in this lesson.

In [15]:
!pip install selenium

  Using cached selenium-3.8.1-py2.py3-none-any.whl


In [63]:
# import
from selenium import webdriver

Selenium requires us to determine a default browser to run. I'm going to opt for Firefox, but Chromium is also a very common choice. http://selenium-python.readthedocs.io/faq.html

In [17]:
# STOP
# what is going to happen when I run the next cell?

In [66]:
# create a driver called Firefox
driver = webdriver.Firefox()

Pretty crazy, right? Let's close that driver.

In [65]:
# close it
driver.close()

In [67]:
# let's boot it up, and visit a URL of our choice
driver = webdriver.Firefox()
driver.get("http://www.python.org")

Awesome. Now we're getting somewhere: programmatically controlling our browser like a human.

Let's return to our problem at hand. We need to visit the OpenTable listing for DC. Once there, we need to get the html to load. In the next cell, prove you can programmatically visit the page.

In [68]:
driver.close()

In [69]:
# visit our OpenTable page
driver = webdriver.Firefox()
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")
# always good to check we've got the page we think we do
assert "OpenTable" in driver.title

In [70]:
driver.title

u'Washington, D.C. Area Restaurants List | OpenTable'

In [71]:
driver.close()

<a id="selenium-js"></a>
### Running JavaScript before scraping

Now, to resolve our JavaScript problem, there's a few things we can do. What I'll do in this case is request that the page load, wait one second, and then I'm going to grab the source html from the page. Because the page should believe I'm visiting from a live connection on a browser client, the JavaScript should render to be a part of the page source. I can then grab the page source.

In [72]:
# import sleep
from time import sleep

In [73]:
# visit our relevant page
driver = webdriver.Firefox()
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")
# wait one second
sleep(1)
#grab the page source
html = driver.page_source

**Pop Quiz:** What do we need to do with this HTML?

In [26]:
# BeautifulSoup it!
html = BeautifulSoup(html, "lxml")

Now, let's return to our earlier problem: How do we locate bookings on the page?

In [27]:
# print out the number bookings for all restaurants
html.find_all('div', {'class':'booking'})

[<div class="booking"><span class="tadpole"></span>Booked 2 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 515 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 285 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 192 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 98 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 135 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 140 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 295 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 57 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 62 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 29 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 54 times today</div>,
 <div class="booking"><span class="

In [28]:
# now print out each booking for the listings using a loop
for entry in html.find_all('div', {'class':'booking'}):
    print entry

<div class="booking"><span class="tadpole"></span>Booked 2 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 515 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 285 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 192 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 98 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 135 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 140 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 295 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 57 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 62 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 29 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 54 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 15

Let's grab just the text of each of these entries.

In [29]:
# do the same as above, but grabbing only the text content
for entry in html.find_all('div', {'class':'booking'}):
    print entry.text

Booked 2 times today
Booked 515 times today
Booked 285 times today
Booked 192 times today
Booked 98 times today
Booked 135 times today
Booked 140 times today
Booked 295 times today
Booked 57 times today
Booked 62 times today
Booked 29 times today
Booked 54 times today
Booked 15 times today
Booked 64 times today
Booked 59 times today
Booked 47 times today
Booked 68 times today
Booked 40 times today
Booked 42 times today
Booked 38 times today
Booked 21 times today
Booked 44 times today
Booked 79 times today
Booked 30 times today
Booked 66 times today
Booked 58 times today
Booked 58 times today
Booked 80 times today
Booked 54 times today
Booked 87 times today
Booked 69 times today
Booked 17 times today
Booked 71 times today
Booked 17 times today
Booked 19 times today
Booked 47 times today
Booked 19 times today
Booked 46 times today
Booked 9 times today
Booked 27 times today
Booked 45 times today
Booked 19 times today
Booked 57 times today
Booked 55 times today
Booked 26 times today
Booked

In [74]:
driver.close()

We've succeeded!

<a id="selenium-regex"></a>
### Using regex to only get digits

But we can clean this up a little bit. We're going to use regular expressions (regex) to grab only the digits that are available in each of the text.

The best way to get good at regex is to, well, just keep trying and testing: http://pythex.org/

In [77]:
# import regex
import re

Given we haven't covered regex, I'll show you how to use the search function to match any given digit.

In [81]:
html = BeautifulSoup(html, "lxml")

In [82]:
# for each entry, grab the text
for booking in html.findAll('div', {'class':'booking'}):
    # match all digits
    match = re.search(r'\d+', booking.text)
    
    if match:
        # print if found
        print match.group()
    else:
        # otherwise pass
        pass

2
507
289
202
94
148
136
305
58
57
30
58
20
65
57
45
66
34
40
37
18
48
81
35
75
57
72
86
53
96
73
16
73
17
22
19
44
41
7
34
53
22
59
49
25
8
32
19
43
84
53
33
31
41
51
29
55
52
9
19
8
32
92
24
64
94
64
17
24
28
9
46
3
11
31
42
11
30
14
12
29
14
26
35
42
43
6
59
6
10
3
42
41
14
30
43
42
15
17


Before we demonstrate all the other amazing things about headless browsers, let's finish up collecting the data we want from this current example. Do you suppose the html parsing we wrote above will still work on the page source we've grabbed from our headless browser?

To be most efficient, we want to only do a single loop for each entry on the page. That means we want to find what element all of other other elements (name, location, price, bookings) is housed within. Where on the page is each entry located?

In [33]:
# print out all entries
#   NOTE: Has many entries. Uncomment the below code to run it!

# soup.find_all('div', {'class':'result content-section-list-row cf with-times'})

Look over the page. Does every single entry have each element we're seeking?
> I did this previously. I know for a fact that not every element has a number of recent bookings. That's probably exactly why OpenTable houses this in JavaScript: they want to continously update the number of bookings with the most relevant number of values.

In [34]:
# What happens when a booking is not available?
# Print out each booking entry, using the identification code we wrote above

for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    print entry.find('div', {'class':'booking'})

<div class="booking"><span class="tadpole"></span>Booked 2 times today</div>
None
<div class="booking"><span class="tadpole"></span>Booked 515 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 285 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 192 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 98 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 135 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 140 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 295 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 57 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 62 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 29 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 54 times today</div>
<div class="booking"><span class="tadpole"></span>Book

In [85]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    try:
        print entry.find('div', {'class':'booking'}).text
    except:
        print('ZERO')

Booked 2 times today
ZERO
Booked 507 times today
Booked 289 times today
Booked 202 times today
Booked 94 times today
Booked 148 times today
Booked 136 times today
Booked 305 times today
Booked 58 times today
Booked 57 times today
Booked 30 times today
Booked 58 times today
Booked 20 times today
Booked 65 times today
Booked 57 times today
Booked 45 times today
Booked 66 times today
Booked 34 times today
Booked 40 times today
Booked 37 times today
Booked 18 times today
Booked 48 times today
Booked 81 times today
Booked 35 times today
Booked 75 times today
Booked 57 times today
Booked 72 times today
Booked 86 times today
Booked 53 times today
Booked 96 times today
Booked 73 times today
Booked 16 times today
Booked 73 times today
Booked 17 times today
Booked 22 times today
Booked 19 times today
Booked 44 times today
Booked 41 times today
Booked 7 times today
Booked 34 times today
Booked 53 times today
Booked 22 times today
Booked 59 times today
Booked 49 times today
Booked 25 times today
B

What do you notice takes the place when booking is not found?

We could use exception handling (`try`/`except` blocks) to resolve this. However, exceptions should only be used to handle rare or unexpected errors -- never for normal program flow.

In this case, we expect that some entries will be zero. So, we can just use an `if` statement that tests whether there are any `divs` present; if not, display `'ZERO'`. Here's a demo:

In [40]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    booking_tag = entry.find('div', {'class':'booking'})
    
    if booking_tag:
        print booking_tag.text
    else:
        print 'ZERO'

Booked 2 times today
ZERO
Booked 515 times today
Booked 285 times today
Booked 192 times today
Booked 98 times today
Booked 135 times today
Booked 140 times today
Booked 295 times today
Booked 57 times today
Booked 62 times today
Booked 29 times today
Booked 54 times today
Booked 15 times today
Booked 64 times today
Booked 59 times today
Booked 47 times today
Booked 68 times today
Booked 40 times today
Booked 42 times today
Booked 38 times today
Booked 21 times today
Booked 44 times today
Booked 79 times today
Booked 30 times today
Booked 66 times today
Booked 58 times today
Booked 58 times today
Booked 80 times today
Booked 54 times today
Booked 87 times today
Booked 69 times today
Booked 17 times today
Booked 71 times today
Booked 17 times today
Booked 19 times today
Booked 47 times today
Booked 19 times today
Booked 46 times today
Booked 9 times today
Booked 27 times today
Booked 45 times today
Booked 19 times today
Booked 57 times today
Booked 55 times today
Booked 26 times today
B

After previously completing this, we observed that all other elements WILL be returned. This means we do not have to always handle these cases.

<a id="challenge-pandas"></a>
### Challenge: Use Pandas to create a DataFrame of bookings

However, the onus is on you to now put all the pieces together.

Loop through each entry. For each entry, grab the relevant information we want (name, location, price, bookings). Produce a dataframe with the columns "name","location","price","bookings" that contains the 100 entries we would like.

In [86]:
import pandas as pd

In [87]:
# I'm going to create my empty df first
dc_eats = pd.DataFrame(columns=["name","location","price","bookings"])

In [43]:
# Put code here that populates the DataFrame using Selenium and BeautifulSoup!

In [44]:
# check out our work
dc_eats.head()

,name,location,price,bookings


Awesome! We succeeded.

<a id="selenium-typing"></a>
### Auto-typing using Selenium

Now, let's explore some of the other functionality of a webdriver. We've barely scratched the surface.

In [92]:
# we can send keys as well
# import
from selenium.webdriver.common.keys import Keys

In [93]:
# open Firefox
driver = webdriver.Firefox()
# visit Python
driver.get("http://www.python.org")
# verify we're in the right place
assert "Python" in driver.title

Let's try automatedly typing `pycon` in the search box and hitting the return key:

In [94]:
# find the search position
elem = driver.find_element_by_name("q")

# clear it
elem.clear()

# type in pycon
elem.send_keys("pycon")

# send those keys
elem.send_keys(Keys.RETURN)

In [95]:
# close
driver.close()

In [96]:
# all at once:
driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title

elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
driver.close()

The above example (and many others) are available in the Selenium docs: http://selenium-python.readthedocs.io/getting-started.html

What is especially important is exploring functionality like locating elements: http://selenium-python.readthedocs.io/locating-elements.html#locating-elements

FAQ:
http://selenium-python.readthedocs.io/faq.html

### Summary

In this lesson, we used the Beautiful Soup library to locate elements on a website then scrape their text. We also used the Selenium headless browser to run JavaScript first before retrieving the page contents.